In [1]:
import numpy as np
import src.solutions.eigenmodes_part1 as eigen_part1
import src.visualizations as vis

In [2]:
N = 5

In [ ]:
initial_matrix = eigen_part1.create_init_matrix_a(N)
print(initial_matrix)

In [ ]:
N = 100

initial_circle = eigen_part1.create_circle(N)
dependency_circle = eigen_part1.create_circle_dependency(N, initial_circle)
initial_matrix = eigen_part1.create_init_matrix_a(N)
print(dependency_circle)

vis.visualize_mesh(initial_matrix)
vis.visualize_mesh(initial_circle)
vis.visualize_mesh(dependency_circle)
